# Deep Convolutional Generative Adversarial Networks (DCGAN)

A model skeleton for Generative Adversarial Networks (GAN)s. It was first introduced in the paper titled *[Generative Adversarial Networks](https://arxiv.org/abs/1406.2661)* by Ian J. Goodfellow, Jean Pouget-Abadie, Mehdi Mirza, Bing Xu, David Warde-Farley, Sherjil Ozair, Aaron Courville, Yoshua Bengio.

> A generative adversarial network (GAN) is a class of machine learning frameworks designed by Ian Goodfellow and his colleagues in June 2014. Two neural networks contest with each other in a game (in the form of a zero-sum game, where one agent's gain is another agent's loss).

> Given a training set, this technique learns to generate new data with the same statistics as the training set. For example, a GAN trained on photographs can generate new photographs that look at least superficially authentic to human observers, having many realistic characteristics. Though originally proposed as a form of generative model for unsupervised learning, GANs have also proved useful for semi-supervised learning, fully supervised learning, and reinforcement learning.

> The core idea of a GAN is based on the "indirect" training through the discriminator, another neural network that can tell how "realistic" the input seems, which itself is also being updated dynamically. This means that the generator is not trained to minimize the distance to a specific image, but rather to fool the discriminator. This enables the model to learn in an unsupervised manner.

> GANs are similar to mimicry in evolutionary biology, with an evolutionary arms race between both networks.

## usage

In [1]:
class norm:
    def fit_transform(self, image):
        return (tf.cast(image, tf.float32) - 127.5) / 127.5
    
    def inverse_transform(self, image):
        return (tf.cast(image, tf.float32) * 127.5) + 127.5

scaler = norm()

In [2]:
import deeply
gan = deeply.hub("dcgan", x = 28, decoder_batch_norm = True, encoder_dropout_rate = 0.3,
                 encoder_layer_growth_rate = 2,
                 init_decoder_units = 256, decoder_layer_growth_rate = 0.5, kernel_size = 5,
                 decoder_strides  = [1, 2], scaler = scaler,
                 final_activation = "tanh")

2022-09-24 17:14:14.503562: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-24 17:14:14.503582: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-24 17:14:16.937646: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-24 17:14:16.937665: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-24 17:14:16.937681: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (test2-cell-collective): /proc/driver/nvidia/version does not exist
2022-09-24 17:14:16.937833: I tensorflow/core/platform/

## example

In [3]:
import tensorflow as tf
import deeply.datasets as dd
from deeply.plots import imgplot as imgplt, mplt
from tqdm.auto import tqdm, trange

mnist  = dd.load("mnist", shuffle_files = True, as_supervised = True)

mapper = lambda image, label: scaler.fit_transform(image)

config = dict(batch_size = 256,
              epochs = 50)

data   = mnist["train"].map(mapper)
batch  = data.batch(config["batch_size"])

In [4]:
noise_dim = gan.generator.input_shape[1]
sample = tf.random.normal([64, noise_dim])

def plot_img(model):
    imgs = model.generator(sample, training = False)
    imgs = scaler.inverse_transform(imgs)
    imgs = tf.cast(imgs, tf.uint8)
    imgplt(imgs, to_file = "gan.png")

In [ ]:
for e in trange(config["epochs"]):
    for b in tqdm(batch):
        history = gan.fit(b, checkpoint_path = "gan.h5d5")
    plot_img(gan)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/235 [00:00<?, ?it/s]

/home/helikarlab/dev/deeply/src/deeply/plots.py:113: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = pplt.subplots(n_plots, 1, sharex = True)


  0%|          | 0/235 [00:00<?, ?it/s]

  0%|          | 0/235 [00:00<?, ?it/s]